In [1]:
import numpy as np
import sympy as sp
import matplotlib.animation as animation 
import matplotlib.pyplot as plt 
from scipy.integrate import odeint,ode
from scipy.optimize import fsolve

In [2]:
l0, l1, l2, w, = sp.symbols('l0,l1,l2,w')

Ibx,Iby,Ibz = sp.symbols('I_bx,I_by,I_bz')
Itx,Ity,Itz = sp.symbols('I_tx,I_ty,I_tz')
Icx,Icy,Icz = sp.symbols('I_cx,I_cy,I_cz')
mb,mt,mc = sp.symbols('m_b,m_t,m_c')

g,P = sp.symbols('g,P')

"""Left leg variables"""
plh, tlh, psilh, tlk, plhd, tlhd, psilhd, tlkd = sp.symbols('phi_lh,theta_lh,psi_lh,theta_lk,phidot_lh,thetadot_lh,psidot_lh,thetadot_lk')
plhddot, tlhddot, psilhddot, tlkddot = sp.symbols('phiddot_lh,thetaddot_lh,psiddot_lh,thetaddot_lk')

"""Right leg varibales"""
prh, trh, psirh, trk, prhd, trhd, psirhd, trkd = sp.symbols('phi_rh,theta_rh,psi_rh,theta_rk,phidot_rh,thetadot_rh,psidot_rh,thetadot_rk')
prhddot, trhddot, psirhddot, trkddot = sp.symbols('phiddot_rh,thetaddot_rh,psiddot_rh,thetaddot_rk')

"""Torso variables"""
p, t, psi, x,y,z, pd, td, psid, xd,yd,zd = sp.symbols('phi,theta,psi,x,y,z,phidot,thetadot,psidot,xdot,ydot,zdot')
pddot, tddot, psiddot, xddot,yddot,zddot = sp.symbols('phiddot,thetaddot,psiddot,xddot,yddot,zddot')

q = sp.Matrix([p, t, psi, x,y,z,plh, tlh, psilh, tlk,prh, trh, psirh, trk])
qdot = sp.Matrix([pd, td, psid, xd,yd,zd , plhd, tlhd, psilhd, tlkd, prhd, trhd, psirhd, trkd])
qddot = sp.Matrix([pddot, tddot, psiddot, xddot,yddot,zddot,plhddot, tlhddot, psilhddot, tlkddot,prhddot, trhddot, psirhddot, trkddot])

def i_1Ti(phi,phid,u,r):
    """
        @phi: Angle of rotation.
        @u : Unit base along which rotation is happening.
        @r : Position vector of joint in world frame.
    """
    
    ux,uy,uz = u
    R = sp.eye(3,3)
    cth = sp.cos(phi)
    sth = sp.sin(phi)
    vth = 1 - cth
    R[0,0] = ux**2*vth + cth
    R[0,1] = ux*uy*vth-uz*sth
    R[0,2] = ux*uz*vth+uy*sth
    R[1,0] = ux*uy*vth + uz*sth
    R[1,1] = uy**2*vth + cth
    R[1,2] = uy*uz*vth - ux*sth
    R[2,0] = ux*uz*vth - uy*sth
    R[2,1] = uy*uz*vth + ux*sth
    R[2,2] = uz**2*vth + cth
    I = sp.eye(3,3)
    
    R1 = R.col_insert(3,(I-R)@r.T)
    tmp = sp.Matrix([0,0,0,1]).T
    
    T = R1.col_join(tmp)
    omega = phid*sp.Matrix(u)
    return T, R, omega

In [3]:
"""For Right Leg """
print('"""For Right Leg """')
T12r,R12r,omega12r = i_1Ti(psirh,psirhd,np.array([0,0,-1]),sp.Matrix([[0,-w,0]]))
T23r,R23r,omega23r = i_1Ti(prh,prhd,np.array([-1,0,0]),sp.Matrix([[0,-w,0]]))
T34r,R34r,omega34r = i_1Ti(trh,trhd,np.array([0,-1,0]),sp.Matrix([[0,-w,0]]))
T45r,R45r,omega45r = i_1Ti(trk,trkd,np.array([0,-1,0]),sp.Matrix([[0,-w,-l1]]))

T14r = T12r@T23r@T34r
T15r = T12r@T23r@T34r@T45r
T15r.shape

"""For Right Leg """


(4, 4)

In [4]:
"""For Left Leg"""
print('"""For Left Leg"""')
T12l,R12l,omega12l = i_1Ti(psilh,psilhd,np.array([0,0,1]),sp.Matrix([[0,w,0]]))
T23l,R23l,omega23l = i_1Ti(plh,plhd,np.array([1,0,0]),sp.Matrix([[0,w,0]]))
T34l,R34l,omega34l = i_1Ti(tlh,tlhd,np.array([0,-1,0]),sp.Matrix([[0,w,0]]))
T45l,R45l,omega45l = i_1Ti(tlk,tlkd,np.array([0,-1,0]),sp.Matrix([[0,w,-l1]]))

T14l = T12l@T23l@T34l
T15l = T12l@T23l@T34l@T45l
T15l.shape

"""For Left Leg"""


(4, 4)

In [5]:
""" For Torso"""

'''World Frame'''
T01 = sp.Matrix([[1, 0, 0, x],
                 [0, 8, 0, y],
                 [0, 0, 1, z], 
                 [0, 0, 0, 1]])
R01 = sp.Matrix([[1,0,0],[0,1,0],[0,0,1]])
T12,R12,omega12 = i_1Ti(p,pd,np.array([1,0,0]),sp.Matrix([[0,0,0]]))
T23,R23,omega23 = i_1Ti(t,td,np.array([0,1,0]),sp.Matrix([[0,0,0]]))
T34,R34,omega34 = i_1Ti(psi,psid,np.array([0,0,1]),sp.Matrix([[0,0,0]]))

T04 = T01@T12@T23@T34
T04.shape

(4, 4)

In [6]:
"""Position vectors for Torso"""
P_h = T04@sp.Matrix([0,0,l0,1])
P_h = sp.Matrix(P_h[:3])
P_h.shape

(3, 1)

In [7]:
"""Position Vectors for Left Leg"""
P_lh = T04@T14l@sp.Matrix([0,w,0,1])
P_lk = T04@T15l@sp.Matrix([0,w,-l1,1])
P_la = T04@T15l@sp.Matrix([0,w,-(l1+l2),1])
P_la.shape

(4, 1)

In [8]:
"""Position Vectors for Right Leg"""
P_rh = T04@T14r@sp.Matrix([0,-w,0,1])
P_rk = T04@T15r@sp.Matrix([0,-w,-l1,1])
P_ra = T04@T15r@sp.Matrix([0,-w,-(l1+l2),1])
P_ra.shape

(4, 1)

In [9]:
"""Position of COM"""
print('"""Position of COM"""')
com_b = T04@sp.Matrix([0,0,0.5*l0,1])
com_lt = T04@T14l@sp.Matrix([0,w,-0.5*l1,1])
com_lc = T04@T14l@sp.Matrix([0,w,-(l1+0.5*l2),1])
com_rt = T04@T14r@sp.Matrix([0,-w,-0.5*l1,1])
com_rc = T04@T14r@sp.Matrix([0,-w,-(l1+0.5*l2),1])
com_b.shape

"""Position of COM"""


(4, 1)

In [10]:
"""Angular velocity in body frame"""

omegaB_2 = omega12
omegaB_3 = omega23 + R23.T@omegaB_2
omegaB_4 = omega34 + R34.T@omegaB_3

"""Angular velocity for left leg"""

omegaB_1l = omega12l + R12l.T@omegaB_4
omegaB_2l = omega23l + R23l.T@omegaB_1l
omegaB_3l = omega34l + R34l.T@omegaB_2l
omegaB_4l = omega45l + R45l.T@omegaB_3l

"""Angular velocity for right leg"""
omegaB_1r = omega12r + R12r.T@omegaB_4
omegaB_2r = omega23r + R23r.T@omegaB_1r
omegaB_3r = omega34r + R34r.T@omegaB_2r
omegaB_4r = omega45r + R45r.T@omegaB_3r
print('"""Angular velocity"""')

"""Angular velocity"""


In [11]:
qdot_diag = sp.diag(pd, td, psid, xd,yd,zd , plhd, tlhd, psilhd, tlkd, prhd, trhd, psirhd, trkd)
qddot_diag = sp.diag(pddot, tddot, psiddot, xddot,yddot,zddot,plhddot, tlhddot, psilhddot, tlkddot,prhddot, trhddot, psirhddot, trkddot)
v_b = com_b.jacobian(q)@qdot
v_lt = com_lt.jacobian(q)@qdot
v_lc = com_lc.jacobian(q)@qdot
v_rt = com_rt.jacobian(q)@qdot
v_rc = com_rc.jacobian(q)@qdot
print('"""linear velocity"""')

"""linear velocity"""


In [12]:
Ib = sp.diag(Ibx, Iby, Ibz)
It = sp.diag(Itx, Ity, Itz)
Ic = sp.diag(Icx, Icy, Icz)


T = 0.5*(mb*(v_b.T@v_b) +mt*(v_rt.T@v_rt) + mc*(v_rc.T@v_rc) + mt*(v_lt.T@v_lt)+ mc*(v_lc.T@v_lc) \
                  +(omegaB_4.T@Ib@omegaB_4) + (omegaB_3l.T@It@omegaB_3l) \
                   +(omegaB_4l.T@Ic@omegaB_4l) + (omegaB_4r.T@Ic@omegaB_4r))

V = sp.Matrix([mb*g*com_b[2] +mt*g*com_rt[2]+ mt*g*com_lt[3]+ mc*g*com_lc[2] + mc*g*com_rc[3]])

L = T - V
L.shape

(1, 1)

In [13]:
dLdqdot = L.jacobian(qdot)
(dLdqdot).shape

(1, 14)

In [14]:
dt_dLdqdot = sp.Matrix.diagonal(dLdqdot.jacobian(q))@qdot_diag + sp.Matrix.diagonal(dLdqdot.jacobian(qdot))@qddot_diag

dt_dLdqdot.shape

(1, 14)

In [15]:
dLdq = L.jacobian(q)
# dLdq.shape

EOM = dt_dLdqdot - dLdq
EOM.shape

(1, 14)

In [16]:
M = EOM.jacobian(qddot)
M.shape

(14, 14)

In [17]:
B = EOM.subs({pddot:0, tddot:0, psiddot:0, xddot:0,yddot:0,zddot:0 , plhddot:0, tlhddot:0, psilhddot:0, tlkddot:0, prhddot:0, trhddot:0, psirhddot:0, trkddot:0})
B.shape

(1, 14)

In [18]:
class traj_gen:
    def __init__(self,t0,tf,q0,qf,q0dot,qfdot,q0ddot,qfddot):
        self.t0 = t0
        self.tf = tf
        self.q0 = q0
        self.qf = qf 
        self.q0dot = q0dot
        self.qfdot = qfdot
        self.q0ddot = q0ddot
        self.qfddot = qfddot
        
    def getb(self):
        t0,tf = self.t0,self.tf
        A = np.array([[1,t0,t0**2,t0**3,t0**4,t0**5],
                       [1,tf,tf**2,tf**3,tf**4,tf**5],
                       [0,1,2*t0,3*t0**2,4*t0**3,5*t0**4],
                       [0,1,2*tf,3*tf**2,4*tf**3,5*tf**4],
                       [0,0,2,6*t0,12*t0**2,20*t0**3],
                       [0,0,2,6*tf,12*tf**2,20*tf**3]])
        C = np.array([[self.q0,self.qf,self.q0dot,self.qfdot,self.q0ddot,self.qfddot]]).T
        B = np.linalg.inv(A)@C
        return B
    
    def get_q(self,t):
        T1 = np.array([[1,t,t**2,t**3,t**4,t**5]])
        T2 = np.array([[0,1,2*t,3*t**2,4*t**3,5*t**4]])
        T3 = np.array([[0,0,2,6*t,12*t**2,20*t**3]])
        B = np.squeeze(self.getb())
        q = B@T1.T
        qdot = B@T2.T
        qddot = B@T3.T
        return q, qdot, qddot

    
# ankle_motion = deg2rad([-7.5 10 10 5 0 -10 -7.5])
# knee_motion = deg2rad([10, -5, 2.5, -10, -10, 15, 10])
# hip_motion = deg2rad([-10, -7.5, -15, 10, 15, 10,-10])

q0 = np.array([0,0,0,0,0,0])
qf = np.array([0,0,0.375,0,0,0])
q0dot = np.array([0,0,0,0,0,0])
qfdot = np.array([0,0,0,0,0,0])
q0ddot = np.array([0,0,0,0,0,0])
qfddot = np.array([0,0,0,0,0,0])
c1  = traj_gen(0,2,q0,qf,q0dot,qfdot,q0ddot,qfddot)
c1.getb()[0]
c1.get_q(1.2)

(array([[0.     ],
        [0.     ],
        [0.25596],
        [0.     ],
        [0.     ],
        [0.     ]]),
 array([[0.   ],
        [0.   ],
        [0.324],
        [0.   ],
        [0.   ],
        [0.   ]]),
 array([[ 0.  ],
        [ 0.  ],
        [-0.27],
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]]))

In [19]:
P_ra1 = sp.Matrix(P_ra[:3])
P_la1 = sp.Matrix(P_la[:3])
J_r = P_ra1.jacobian(q)
J_l = P_la1.jacobian(q)

Jdot_l = (sp.Matrix([J_l.row(0)]).jacobian(q)@qdot).T
Jdot_r = (sp.Matrix([J_r.row(0)]).jacobian(q)@qdot).T

for i in range(1,3):
    Jdot_l = Jdot_l.row_insert(i,(sp.Matrix([J_l.row(i)]).jacobian(q)@qdot).T)
    Jdot_r = Jdot_r.row_insert(i,(sp.Matrix([J_r.row(i)]).jacobian(q)@qdot).T)
     
Jdot_l.shape

(3, 14)

In [35]:
class parameters:
    def __init__(self):
        self.l0, self.l1,self.l2, self.w = 1, 0.5, 0.5, 0.1
        self.mb, self.mt,self.mc = 70,10,5
        self.Ibx,self.Iby,self.Ibz = 5,3,2
        self.Itx, self.Ity, self.Itz = 1,0.3,2
        self.Icx,self.Icy,self.Icz = 0.5,0.15,1
        """ torso params"""
        self.p, self.t, self.psi = 0,0,0
        self.pd, self.td, self.psid = 0,0.5,0
        self.x,self.y,self.z = 0,0,0
        self.xd,self.yd,self.zd = 0,0,0
        """left leg params"""
        self.plh, self.tlh, self.psilh, self.tlk = 0,0,0,0
        self.plhd, self.tlhd, self.psilhd, self.tlkd = 0,0,0,0
        """right leg params"""
        self.prh, self.trh, self.psirh, self.trk = 0,0,0,0
        self.prhd, self.trhd, self.psirhd, self.trkd = 0, 0, 0, 0 
        
        
        
p1 = parameters()


class Humanoid(parameters):
    def __init__(self):
        super().__init__()
        self.Z0 = np.array([self.p, self.t, self.psi,\
                            self.pd, self.td, self.psid,\
                            self.plh, self.tlh, self.psilh, self.tlk,\
                            self.plhd, self.tlhd, self.psilhd, self.tlkd,\
                            self.prh, self.trh, self.psirh, self.trk,\
                            self.prhd, self.trhd, self.psirhd, self.trkd\
                           ])
        
    def set_equations(self,M,B,J_r,J_l,Jdot_r,Jdot_l):
        M = M.subs({l0:self.l0, l1:self.l1,l2:self.l2, w:self.w,\
                    mb:self.mb, mt:self.mt,mc:self.mc,\
                    Ibx:self.Ibx,Iby:self.Iby,Ibz:self.Ibz,\
                    Itx:self.Itx,Ity:self.Ity,Itz:self.Itz,\
                    Icx:self.Icx,Icy:self.Icy,Icz:self.Icz})
        
        B = B.subs({l0:self.l0, l1:self.l1,l2:self.l2, w:self.w,\
                    mb:self.mb, mt:self.mt,mc:self.mc,\
                    Ibx:self.Ibx,Iby:self.Iby,Ibz:self.Ibz,\
                    Itx:self.Itx,Ity:self.Ity,Itz:self.Itz,\
                    Icx:self.Icx,Icy:self.Icy,Icz:self.Icz})
        
        J_r = J_r.subs({l1:self.l1,l2:self.l2, w:self.w})
        J_l = J_l.subs({l1:self.l1,l2:self.l2, w:self.w})
        
        Jdot_r = Jdot_r.subs({l1:self.l1,l2:self.l2, w:self.w})
        Jdot_l = Jdot_l.subs({l1:self.l1,l2:self.l2, w:self.w})
        
#         print(Jdot_r)
        
        
        self.M_num = sp.lambdify([p, t, psi,plh, tlh, psilh, tlk,\
                             prh, trh, psirh, trk],M)
        
        self.B_num = sp.lambdify([p, t, psi,plh, tlh, psilh, tlk,                           
                                  prh, trh, psirh, trk,
                                  pd,td,psid,plhd, tlhd, psilhd, tlkd,                           
                                  prhd, trhd, psirhd, trkd], B)
        
        self.J_r_num = sp.lambdify([p, t, psi,prh, trh, psirh, trk], J_r)
        self.J_l_num = sp.lambdify([p, t, psi,plh, tlh, psilh, tlk], J_l)
        
        self.Jdot_r_num = sp.lambdify([p, t, psi,  pd, td, psid,  prh, trh, psirh, trk,  prhd, trhd, psirhd, trkd],Jdot_r)
        
        self.Jdot_l_num = sp.lambdify([p, t, psi,  pd, td, psid,  plh, tlh, psilh, tlk,  plhd,tlhd, psilhd, tlkd],Jdot_l)
        
        print(self.Jdot_l_num(0.3,0.2,0.2, 0.3,0.2,0.2, 0.5,0.1,0.15,0.15, 0.5,0.1,0.15,0.15)) 
        
    def get_T(self,t,leg, A_, b_):
        
        """Gives BT part of the equation"""
        """A_,b_ are Estimated parameters of the system"""
        B1 = np.zeros((6,8))
        B2 = np.eye((8))
        B3 = np.zeros((3,8))
        B = np.concatenate((B1,B2,B3))
        
        S_L = np.array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])
        
        S_R = np.array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])
        
        S = S_L if leg == "left" else S_R
            
        qddot_des, qdot_des, q_des = c1.get_q(t)
        
        """TODO: add trajectory start and end points"""
        Kd = np.eye((8))
        Kp = np.eye((8))
        
        print(qddot_des.shape)
        print((qdot_des - qdot).shape)
        v = qddot_des + Kd@(qdot_des - qdot) + Kp@(q_des - q) 

        T = B@np.linalg.inv(S@np.linalg.inv(A_)@b_)@(v - S@np.linalg.inv(A_)@b_)
            
        return T
        
        
    def rhs(self,t,z,leg):
        self.M_temp = self.M_num(z[0],z[1],z[2],z[3],z[4],z[5],z[6],z[7],z[8],z[9],z[10])
        self.B_temp = self.B_num(z[0],z[1],z[2],z[3],z[4],z[5],z[6],z[7],z[8],z[9],z[10],z[11],z[12],z[13],z[14],z[15],z[16],z[17],z[18],z[19],z[20],z[21])
        
        
        self.Jdot_r_temp = self.Jdot_r_num(z[0],z[1],z[2],z[3],z[4],z[5],z[14],z[15],z[16],z[17],z[18],z[19],z[20],z[21])
        self.Jdot_l_temp = self.Jdot_l_num(z[0],z[1],z[2],z[3],z[4],z[5],z[6],z[7],z[8],z[9],z[10],z[11],z[12],z[13])
        self.J_l_temp = self.J_l_num(z[0],z[1],z[2],z[6],z[7],z[8],z[9])
        self.J_r_temp = self.J_r_num(z[0],z[1],z[2],z[14],z[15],z[16],z[17])
        
        if leg == "left":
            A = np.array([[M,        -self.J_l_temp.T],
                          [self.J_l_temp, np.zeros(3)]
                         ])
            N = np.array([[self.get_T(t,leg,self.M_temp,self.B_temp) - self.B_temp],
                         [-self.Jdot_l_temp@qdot]])
            
        elif leg == "right":
            A = np.array([[M,        -self.J_r_temp.T],
                          [self.J_r_temp, np.zeros(3)]
                         ])
            N = np.array([[self.get_T(t,leg,self.M_temp,self.B_temp) - self.B_temp],
                          [-self.Jdot_r_temp@qdot]])
        else:
            print("ERROR: stance leg not set.")
            
        zdot = np.linalg.inv(A)@N
        print(zdot)
        
        return zdot
    
    
    def one_step(self,z):
        self.z = z
        eqn = ode(self.rhs)
        eqn.set_initial_value(self.z,0).set_f_params(leg)
        dt = 0.01
#         x_start = self.Z[-1][4] + self.l*np.sin(self.z[0])
#         #y_start = self.l*np.sin(theta1)
        while eqn.successful() and self.collision() and (self.t_end - eqn.t) > 0:
            self.z = eqn.integrate(eqn.t+dt)
            z_temp = np.concatenate(([self.z],[[x_start - self.l*np.sin(self.z[0]), self.l*np.cos(self.z[0])]]),axis=None)
            self.Z = np.append(self.Z,[z_temp],axis=0)
        
        self.z = self.foot_strike()
        return self.z
        
        
    def foot_strike(self):
        
        A = np.array([[M, -J_sw],[J_sw,np.zeros(3)]])
        
        N = np.array([[J_st.T@P, M@z],np.zeros(14)])
        
        After_collision = np.linalg.inv(A)@N
        
        return After_collision
    
    def start_walker(self,steps,z):
        i = 0 
        self.z = z0
        while (steps > i):
            self.z = self.one_step(self.z)
            i += 1
        return self.z
    
h1 = Humanoid()
# h1.get_T()
h1.set_equations(M,B,J_r,J_l,Jdot_r,Jdot_l)
h1.rhs(0,h1.Z0,leg="left")

[[ 0.00000000e+00  3.34472651e-01 -4.11097269e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -1.78085369e-03 -5.50128839e-02
  -4.18885636e-01 -1.09758282e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.57703944e+00 -1.11479322e+00 -2.09603239e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -5.61452072e+00  1.39316135e+00
  -1.87212224e+00  3.17672869e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.12955667e-01  2.58569578e-01  1.28582899e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.42406197e-01  5.92898540e-02
   1.24197121e-01  2.95786672e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
(6, 1)


/tmp/ipykernel_5298/1395188140.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  A = np.array([[M,        -self.J_l_temp.T],


ShapeError: Matrix size mismatch: (14, 1) + (6, 1)

In [21]:
class Animate():
    def __init__(self,q):
        self.axis = plt.axes(autoscale_on=False,xlim = (-0.5, 2),ylim = (-0.1, 2),zlim = ( -0.1,2)) 
        self.joint_data = q
        self.head1, = self.axis.plot(0,0,0, marker='o') 
        self.stem1, = self.axis.plot([],[],[], lw = 2)
        
        self.kneer, = self.axis.plot(0,0,0, marker='o')
        self.kneel, = self.axis.plot(0,0,0, marker='o')

        self.body, = self.axis.plot([],[],[], lw = 2)
    
    def initiate(self):
        self.head1.set_data([],[],[]) 
        self.stem1.set_data([],[],[])
        
        self.kneer.set_data([],[],[])
        self.kneel.set_data([],[],[])
        
        self.body.set_data([],[],[])
        
        return self.head1,self.stem1,self.kneel,self.kneer,self.body
    
    def animate(self,i):
        x,y,z,p,t,psi,plh, tlh, psilh, tlk,prh, trh, psirh, trk = self.joint_data[i]
        
        self.head_pos = P_h.subs(x,y,z,p,t,psid)
        
        self.head1.set_data(self.head_pos[0],selff.head_pos[1],self.head_pos[2])
        
        self.stem1.set_data([x, self.head_pos[0]],[y, self.head_pos[1]], [z, self.head_pos[2]])
        
        self.rhip_pos = P_rh.subs(x,y,z,p,t,psi,plh,prh, trh, psirh, trk)
        self.rknee_pos = P_rk.subs(x,y,z,p,t,psi,plh,prh, trh, psirh, trk)
        self.rankle_pos = P_ra.subs(x,y,z,p,t,psi,plh,prh, trh, psirh, trk)
        
        self.lhip_pos = P_lh.subs(x,y,z,p,t,psi,plh,plh, tlh, psilh, tlk)
        self.lknee_pos = P_lk.subs(x,y,z,p,t,psi,plh,plh, tlh, psilh, tlk)
        self.lankle_pos = P_la.subs(x,y,z,p,t,psi,plh,plh, tlh, psilh, tlk)
        
        self.body.set_data([self.rankle_pos[0],self.rknee_pos[0],self.rhip_pos[0],x,self.lhip_pos[0],self.lknee_pos[0],self.lankle_pos[0]],
                          [self.rankle_pos[1],self.rknee_pos[1],self.rhip_pos[1],y,self.lhip_pos[1],self.lknee_pos[1],self.lankle_pos[1]],
                          [self.rankle_pos[2],self.rknee_pos[2],self.rhip_pos[2],z,self.lhip_pos[2],self.lknee_pos[2],self.lankle_pos[2]])
        
        return self.head1, self.stem1, self.body,
    
    def save_video(file_name = "3d_bipedal_walk"):
        anim = animation.FuncAnimation(self.fig, self.animate, init_func = self.initiate, 
                               frames = len(self.X_n)-1, interval = 10, blit = True) 
        anim.save(file_name, writer = 'ffmpeg', fps = 200)
        
        
    